In [1]:
import numpy as np
import pandas as pd
import os
%pwd

'c:\\Users\\nohsoocheol.RNDIT\\Downloads\\99. Etc\\testpy\\global-wheat-detection\\Kaggle-Global-Wheat-Detection'

In [2]:
import random
random.seed(0)
np.random.seed(0)

In [3]:
train = pd.read_csv('../input/global-wheat-detection/train.csv')
submission = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [4]:
from tqdm import tqdm
train_source = train[['image_id', 'source']]
train_source = train_source.drop_duplicates()
train_source = train_source.reset_index(drop=True)
train_source['bboxs'] = np.nan
for i in tqdm(range(train_source.shape[0])):
    train_source.loc[i, 'bboxs'] = train['image_id'][train['image_id']==train_source.loc[i, 'image_id']].count().astype(np.int64)
train_source['bboxs'] = train_source['bboxs'].astype(int)
train_source.head()

100%|██████████| 3373/3373 [00:49<00:00, 67.85it/s]


,image_id,source,bboxs
0,b6ab77fd7,usask_1,47
1,b53afdf5c,usask_1,46
2,7b72ea0fb,usask_1,41
3,91c9d9c38,usask_1,33
4,41c0123cc,usask_1,34


In [5]:
from sklearn.model_selection import StratifiedKFold
train_source_y = train_source['source']

search_rand = False
search_rand_range = 4000
opt_diff = 31
opt_rand = 3277
if search_rand:
    for rand_s in tqdm(range(search_rand_range)):
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_s)
        fold_indices = []
        for imgid in skf.split(train_source, train_source_y):
            fold_indices.append(imgid)

        train_bbox_sum = []
        valid_bbox_sum = []
        for i in range(5):
            train_bbox_sum.append(train_source.loc[fold_indices[i][0], 'bboxs'].sum())
            valid_bbox_sum.append(train_source.loc[fold_indices[i][1], 'bboxs'].sum())

        train_diff = np.max(train_bbox_sum) - np.min(train_bbox_sum)
        if opt_diff > train_diff:
            opt_diff = train_diff
            opt_rand = rand_s
    print('opt_diff: ', opt_diff)
    print('opt_rand: ', opt_rand)

In [6]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=opt_rand)
fold_indices = []
for imgid in skf.split(train_source, train_source_y):
    fold_indices.append(imgid)

In [7]:
train_bbox = train['bbox'].str.lstrip('[').str.rstrip(']').str.split(', ', expand=True).astype(float)
train_bbox.columns=['x','y','w','h']
train_bbox['image_id'] = train['image_id']
train_bbox['class'] = 0
train_bbox.head()

,x,y,w,h,image_id,class
0,834.0,222.0,56.0,36.0,b6ab77fd7,0
1,226.0,548.0,130.0,58.0,b6ab77fd7,0
2,377.0,504.0,74.0,160.0,b6ab77fd7,0
3,834.0,95.0,109.0,107.0,b6ab77fd7,0
4,26.0,144.0,124.0,117.0,b6ab77fd7,0


In [8]:
import shutil
def clean_dir(path_dir):
    if os.path.exists(path_dir):
        shutil.rmtree(path_dir)
    os.mkdir(path_dir)

In [9]:
TRAIN_ORG = '../input/global-wheat-detection/train/'
TEST_ORG = '../input/global-wheat-detection/test/'

In [10]:
import json
class CocoJson():
    def __init__(self):
        wheat = {}
        wheat['id'] = 1
        wheat['name'] = 'wheat'
        wheat['supercategory'] = 'None'

        self.json = {}
        self.json['info'] = {}
        self.json['licenses'] = []
        self.json['categories'] = [wheat]
        self.json['images'] = []
        self.json['annotations'] = []
    
    def add_image(self, id, name, width=1024, height=1024):
        image = {}
        image['id'] = id
        image['file_name'] = name
        image['width'] = width
        image['height'] = height
        self.json['images'].append(image)

    def add_annot(self, id, image_id, bbox):
        annot = {}
        annot['id'] = id
        annot['image_id'] = image_id
        annot['category_id'] = 1
        annot['iscrowd'] = 0
        annot['bbox'] = bbox
        self.json['annotations'].append(annot)

In [11]:
for fold in [0]:
    print('##### do ' + str(fold) + '-th fold #####')
    SPLIT_PATH = '../split-fold' + str(fold) + '/'
    TRAIN_PATH = SPLIT_PATH + 'train/'
    VALID_PATH = SPLIT_PATH + 'valid/'
    ANNOT_PATH = SPLIT_PATH + 'annotations/'
    clean_dir(SPLIT_PATH)
    clean_dir(TRAIN_PATH)
    clean_dir(VALID_PATH)
    clean_dir(ANNOT_PATH)

    train_json = CocoJson()
    annot_id = 0
    for image_id, idx in enumerate(tqdm(fold_indices[fold][0])):
        name = train_source.loc[idx, 'image_id']
        shutil.copy(TRAIN_ORG + name + '.jpg', TRAIN_PATH)
        train_json.add_image(image_id, name+'.jpg')
        annot_list = train_bbox[train_bbox['image_id'] == name].iloc[:, 0:4]
        for i in range(annot_list.shape[0]):
            train_json.add_annot(annot_id, image_id, annot_list.iloc[i].tolist())
            annot_id = annot_id+1
    with open(ANNOT_PATH + 'instances_train.json', 'w') as json_file:
        json.dump(train_json.json, json_file)

    valid_json = CocoJson()
    annot_id = 0
    for image_id, idx in enumerate(tqdm(fold_indices[fold][1])):
        name = train_source.loc[idx, 'image_id']
        shutil.copy(TRAIN_ORG + name + '.jpg', VALID_PATH)
        valid_json.add_image(image_id, name+'.jpg')
        annot_list = train_bbox[train_bbox['image_id'] == name].iloc[:, 0:4]
        for i in range(annot_list.shape[0]):
            valid_json.add_annot(annot_id, image_id, annot_list.iloc[i].tolist())
            annot_id = annot_id+1
    with open(ANNOT_PATH + 'instances_valid.json', 'w') as json_file:
        json.dump(valid_json.json, json_file)

##### do 0-th fold #####
100%|██████████| 677/677 [00:19<00:00, 34.07it/s]


In [13]:
PROJ_PATH = SPLIT_PATH + 'projects/'
clean_dir(PROJ_PATH)
shutil.copy('./efficientdetconfig/split-fold0.yml', PROJ_PATH)

'../split-fold0/projects/split-fold0.yml'